## Supplement to Proposition 6.14

In Proposition 6.14, we produce an example of a $w \in T\mathbb{S}_8^{\circ}$ where the closed immersion in Theorem 1.1 is not an isomorphism. In this notebook, we record all the computations appearing in the proof. This works with Oscar 0.11.3. 

**Note** this notebook is `1`-indexed, whereas the paper is `0`-indexed. 

In [1]:
using Oscar
using Combinatorics
pm = Polymake

cd("..")
currentDir = pwd()
include(joinpath(currentDir, "src/pfaffian_functions.jl"))

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.10.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The Oscar Development Team


twist (generic function with 1 method)

Let $M_{\text{oct}}$ be the $(4,8)$-matroid with nonbases
\begin{equation}
1234, 1256, 1278, 1357, 1368, 1458, 2367, 2457, 2468, 3456, 3478, 5678.
\end{equation}
The 8 supporting hyperplanes of the octahedron in $\mathbb{P}^{3}$ is a realization of this matroid. We record this matroid as `Moct`, and its nonbases as `Boct`.  

In [2]:
# The normal vectors to the supporting hyperplanes of the standard octahedron
A = Matrix{Int64}(
    [1 -1 -1 -1;
     1 1 -1 -1;
     1 -1 1 -1;
     1 1 1 -1;
     1 -1 -1 1;
     1 1 -1 1;
     1 -1 1 1;
     1 1 1 1])

c48 = collect(powerset(1:8, 4, 4))

Boct = [a for a in c48 if det(A[a, 1:4]) == 0]
Moct = matroid_from_nonbases(Boct, 8)

Matroid of rank 4 on 8 elements

Now, let $w$ be the corank vector of $M_{\text{oct}}$, viewed as an even $\Delta$-matroid. This is the vector in $\mathbb{R}^{E(8)}\cong \mathbb{R}^{128}$ given by
\begin{equation*}
w_{\lambda} = 4 \; \text{ if } \; |\lambda| = 0, 8, 
\hspace{10pt} 
w_{\lambda} = 2 \text{ if } |\lambda| = 2, 6,
\hspace{10pt} 
w_{\lambda} = 1 \text{ if }  \lambda \in \textstyle{\binom{[8]}{4}} \setminus B_{\text{oct}},
\hspace{10pt} 
w_{\lambda} = 0 \text{ if }  \lambda \in B_{\text{oct}}.
\end{equation*}

In [3]:
even = [a for a in collect(powerset(1:8)) if length(a)%2 == 0]

w = zeros(Int64, 128)
for i in 1:128
    if (length(even[i]) == 0 ||  length(even[i]) == 8)
        w[i] = 4
    elseif  (length(even[i]) == 2 ||  length(even[i]) == 6)
        w[i] = 2
    elseif even[i] in Boct
        w[i] = 1
    end
end

We compute the regular subdivision of $\Delta(8)$ with respect to $w$. Here,

- `Delta`  $ = \Delta(8)$
- `SOP`  $ = \Delta_{w}$ the subdivision.
- `MCs` $ = $ the maximal cells.
- `G` = $\Gamma_{w}$ the dual graph.
- `Ms` = a list of the even $\Delta$-matroids of the maximal cells.

In [4]:
Delta = hcat([indicator_vector(a, 8) for a in even]...)
SOP = SubdivisionOfPoints(Delta', w)
C = secondary_cone(SOP)
MCs = maximal_cells(SOP)
G = SOP.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH;
Ms =  [[even[i] for i in MCs[j]] for j in 1:length(MCs)];

Run the following cell to visualize the graph `G`. 

In [5]:
#pm.visual(G)

Next, let's translate the labeling of the maximal cells in `Ms` to the notation in the paper. 

In [6]:
comb4_8 = collect(powerset(1:8, 4, 4))
C1 = [x for x in even if x in setdiff(comb4_8, Boct) || (length(x) < 4)]
C2 = [x for x in even if x in setdiff(comb4_8, Boct) || (length(x) > 4)]


function A_eta(eta)
    return [x for x in even if 
                (length(x) == 4 && length(setdiff(x,eta))<= 1) || 
                (length(x) == 2 && is_subset(x,eta))
            ]
end

function D_eta(eta)
    return [x for x in even if 
                (length(x) == 4 && length(setdiff(x,eta))<= 1) || 
                (length(x) == 6 && is_subset(eta,x))
            ]
end

all([
C2 == Ms[1],
D_eta([1, 2, 3, 4]) == Ms[2],
D_eta([3, 4, 5, 6]) == Ms[3],
D_eta([1, 2, 5, 6]) == Ms[4],
D_eta([2, 4, 6, 8]) == Ms[5],
A_eta([1, 2, 5, 6]) == Ms[6],
A_eta([1, 3, 6, 8]) == Ms[7],
A_eta([2, 3, 6, 7]) == Ms[8],
A_eta([1, 2, 7, 8]) == Ms[9],
D_eta([1, 2, 7, 8]) == Ms[10],
A_eta([1, 3, 5, 7]) == Ms[11],
A_eta([2, 4, 6, 8]) == Ms[12],
D_eta([5, 6, 7, 8]) == Ms[13],
D_eta([2, 4, 5, 7]) == Ms[14],
D_eta([1, 3, 5, 7]) == Ms[15],
D_eta([1, 4, 5, 8]) == Ms[16],
D_eta([2, 3, 6, 7]) == Ms[17],
D_eta([3, 4, 7, 8]) == Ms[18],
A_eta([1, 4, 5, 8]) == Ms[19],
A_eta([5, 6, 7, 8]) == Ms[20],
A_eta([3, 4, 5, 6]) == Ms[21],
A_eta([3, 4, 7, 8]) == Ms[22],
A_eta([2, 4, 5, 7]) == Ms[23],
C2 == Ms[1],
A_eta([1, 2, 3, 4]) == Ms[25],
D_eta([1, 3, 6, 8]) == Ms[26]
        ])

true

**Claim 1.** The morphisms $\mathbb{S}_{D_{\eta}} \to \mathbb{S}_{D_{\eta}'} \times \mathbb{S}_{D_{\eta}''}$ are closed immersions.

We consider the case $\eta = 1234$. We twist $D_{\eta}, D_{\eta}', D_{\eta}''$ by $4678$. The following computation verifies that the presentation of the coordinate rings of the corresponding spinor strata are as in the text. 

In [7]:
D1234 = D_eta([1,2,3,4]); # D_eta
Dp1234 = intersect(D1234, A_eta([1,2,3,4])); # D'_eta
Dpp1234 = intersect(D1234, C2); # D"_eta
Dppp1234 = intersect(Dp1234, Dpp1234)

tD1234 = twist(D1234, [4,6,7,8]);
tDp1234 = twist(Dp1234, [4,6,7,8]);
tDpp1234 = twist(Dpp1234, [4,6,7,8]);
tDppp1234 = twist(Dppp1234, [4,6,7,8]);

(ID,SD,RD,xD) = even_Delta_matroid_stratum(tD1234, 8, QQ);
(IDp,SDp,RDp,xDp) = even_Delta_matroid_stratum(tDp1234, 8, QQ);
(IDpp,SDpp,RDpp,xDpp) = even_Delta_matroid_stratum(tDpp1234, 8, QQ);
(IDppp,SDppp,RDppp,xDppp) = even_Delta_matroid_stratum(tDppp1234, 8, QQ);

In [8]:
println("Polynomial ring generators")
@show gens(RD);
@show gens(RDp);
@show gens(RDpp); 
@show gens(RDppp); print("\n")

println("Ideals")
@show ID;
@show IDp;
@show IDpp; 
@show IDppp; print("\n")

println("Semigroups")
@show SD;
@show SDp;
@show SDpp;
@show SDppp; print("\n")


Polynomial ring generators
gens(RD) = fmpq_mpoly[x_{1, 4}, x_{2, 4}, x_{3, 4}, x_{4, 5}, x_{4, 6}, x_{5, 6}, x_{4, 7}, x_{5, 7}, x_{4, 8}, x_{5, 8}]
gens(RDp) = fmpq_mpoly[x_{1, 4}, x_{2, 4}, x_{3, 4}, x_{4, 5}, x_{5, 6}, x_{5, 7}, x_{5, 8}]
gens(RDpp) = fmpq_mpoly[x_{1, 4}, x_{2, 4}, x_{3, 4}, x_{4, 6}, x_{5, 6}, x_{4, 7}, x_{5, 7}, x_{4, 8}, x_{5, 8}]
gens(RDppp) = fmpq_mpoly[x_{1, 4}, x_{2, 4}, x_{3, 4}, x_{5, 6}, x_{5, 7}, x_{5, 8}]

Ideals
ID = fmpq_mpoly[0]
IDp = fmpq_mpoly[0]
IDpp = fmpq_mpoly[0]
IDppp = fmpq_mpoly[0]

Semigroups
SD = fmpq_mpoly[1, x_{5, 8}, x_{5, 7}, x_{5, 6}, x_{4, 8}, x_{4, 7}, x_{4, 6}, -x_{4, 7}*x_{5, 8} + x_{5, 7}*x_{4, 8}, x_{4, 5}, -x_{4, 6}*x_{5, 8} + x_{5, 6}*x_{4, 8}, -x_{4, 6}*x_{5, 7} + x_{5, 6}*x_{4, 7}, x_{3, 4}, x_{3, 4}*x_{5, 8}, x_{3, 4}*x_{5, 7}, x_{3, 4}*x_{5, 6}, x_{2, 4}, x_{2, 4}*x_{5, 8}, x_{2, 4}*x_{5, 7}, x_{2, 4}*x_{5, 6}, x_{1, 4}, x_{1, 4}*x_{5, 8}, x_{1, 4}*x_{5, 7}, x_{1, 4}*x_{5, 6}]
SDp = fmpq_mpoly[1, x_{5, 8}, x_{5, 7}, x_{5, 6

**Claim 2.** The morphisms $\mathbb{S}_{B_{\eta}} \to \mathbb{S}_{B_{\eta}''}$ are SDC-morphisms.

Let's verify that $D_{1234}$ is obtained from $B_{1234}$ by first twisting $B_{1234}$ by $[8]$ then applying the permutation $(15)(26)(37)(48)$. 

In [9]:
B1234 = A_eta([1,2,3,4]);
tB1234 = twist(B1234, [1,2,3,4,5,6,7,8])

p = @perm (1,5)(2,6)(3,7)(4,8)
ptB1234 = sort!([sort!(p.(x)) for x in tB1234]) 

sort(ptB1234) == sort(D1234)

true

**Claim 3.** 	The fiber product $\mathbb{S}_{C_1}\times_{\mathbb{S}_{C_{12}}} \mathbb{S}_{C_2}$ is smooth and irreducible of dimension $17$. 


In [10]:
tC1 = twist(C1, [4,6,7,8])
tC2 = twist(C2, [4,6,7,8])
tC12 = twist(intersect(C1,C2), [4,6,7,8])

(IC1,SC1,RC1,xC1) = even_Delta_matroid_stratum(tC1, 8, QQ);
(IC2,SC2,RC2,xC2) = even_Delta_matroid_stratum(tC2, 8, QQ);
(IC12,SC12,RC12,xC12) = even_Delta_matroid_stratum(tC12, 8, QQ);

In [11]:
println("Polynomial ring generators")
@show gens(RC1);
@show gens(RC2);
@show gens(RC12); print("\n")

println("Ideals")
@show IC1;
@show IC2;
@show IC12; print("\n")

Polynomial ring generators
gens(RC1) = fmpq_mpoly[x_{1, 2}, x_{1, 3}, x_{2, 3}, x_{1, 4}, x_{2, 4}, x_{3, 4}, x_{1, 5}, x_{2, 5}, x_{3, 5}, x_{1, 6}, x_{2, 6}, x_{5, 6}, x_{1, 7}, x_{3, 7}, x_{5, 7}, x_{1, 8}, x_{2, 8}, x_{3, 8}, x_{5, 8}]
gens(RC2) = fmpq_mpoly[x_{1, 4}, x_{2, 4}, x_{3, 4}, x_{1, 6}, x_{2, 6}, x_{4, 6}, x_{5, 6}, x_{1, 7}, x_{3, 7}, x_{4, 7}, x_{5, 7}, x_{6, 7}, x_{1, 8}, x_{2, 8}, x_{3, 8}, x_{4, 8}, x_{5, 8}, x_{6, 8}, x_{7, 8}]
gens(RC12) = fmpq_mpoly[x_{1, 4}, x_{2, 4}, x_{3, 4}, x_{1, 6}, x_{2, 6}, x_{5, 6}, x_{1, 7}, x_{3, 7}, x_{5, 7}, x_{1, 8}, x_{2, 8}, x_{3, 8}, x_{5, 8}]

Ideals
IC1 = fmpq_mpoly[0, -x_{1, 4}*x_{5, 6}*x_{3, 8} - x_{3, 4}*x_{1, 6}*x_{5, 8} + x_{3, 4}*x_{5, 6}*x_{1, 8}, -x_{1, 4}*x_{5, 7}*x_{2, 8} - x_{2, 4}*x_{1, 7}*x_{5, 8} + x_{2, 4}*x_{5, 7}*x_{1, 8}, x_{1, 6}*x_{3, 7}*x_{2, 8} + x_{2, 6}*x_{1, 7}*x_{3, 8} - x_{2, 6}*x_{3, 7}*x_{1, 8}, -x_{3, 7}*x_{5, 8} + x_{5, 7}*x_{3, 8}, -x_{2, 6}*x_{5, 8} + x_{5, 6}*x_{2, 8}, -x_{2, 4}*x_{3, 8} + x_{3

Now we perform the substitution outlined in the paper. 

In [12]:
Rnew, (x24,x56,x17,x57,x18,x28,x38,x58) = 
    PolynomialRing(QQ, ["x24", "x56", "x17", "x57", "x18", "x28", "x38", "x58"]);
FRnew = FractionField(Rnew);
c = [x18*x24//(2*x28), x24, x24*x38//x28, x18*x56//(2*x58), x56*x28//x58, 
     x56, x18*x57//(2*x58), x57*x38//x58, x57,x18,x28,x38,x58];
phi = hom(RC12, FractionField(Rnew), a->a, c);

In [13]:
all([f == FRnew(0) for f in phi.(IC12)])

true